In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re
from datetime import datetime

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import urllib.request


def get_html(url):
    req = urllib.request.Request(
        url,
        data=None,
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        }
    )
    return urllib.request.urlopen(req).read()

In [2]:
import sys
sys.path.append(os.path.join(sys.path[0], "../..") )
from tools.functions import save_file

In [3]:
def get_html_rec(url, max_depth=1, curr_depth=1):
    try:
        html = get_html(url)
        return html
    except UnicodeDecodeError:
        print("UnicodeDecodeError", url)
        sleep(10)
        if curr_depth < max_depth:
            html = get_html_rec(url, max_depth, curr_depth+1)
            return html
        print("Не удалось")
        raise

In [4]:
prev_years = pd.read_html("https://abit.itmo.ru/pass_rate/", header=0)[0]
prev_years.columns = ['Шифр', 'Направление подготовки (специальность)', 'Предметы',
       'Проходной балл (ЕГЭ+ИД) 2017', 'Средний балл (ЕГЭ) 2017', 
       'Проходной балл (ЕГЭ+ИД) 2018', 'Средний балл (ЕГЭ) 2018', 
       'Проходной балл (ЕГЭ+ИД) 2019', 'Средний балл (ЕГЭ) 2019', 
    ]

In [5]:
html = get_html("https://abit.itmo.ru/bachelor/rating_rank/all/")
page = BS(html)

fac_list = {}
for fac in page.select_one(".static-page-rule > div:nth-child(1) > ul:nth-child(2)").select("li > a"):
    fac_list[fac.text] = "https://abit.itmo.ru" + fac.attrs['href']
fac_list

{'01.03.02 «Прикладная математика и информатика»': 'https://abit.itmo.ru/bachelor/rating_rank/all/261/',
 '09.03.01 «Информатика и вычислительная техника»': 'https://abit.itmo.ru/bachelor/rating_rank/all/262/',
 '09.03.02 «Информационные системы и технологии»': 'https://abit.itmo.ru/bachelor/rating_rank/all/263/',
 '09.03.03 «Прикладная информатика»': 'https://abit.itmo.ru/bachelor/rating_rank/all/264/',
 '09.03.04 «Программная инженерия»': 'https://abit.itmo.ru/bachelor/rating_rank/all/265/',
 '10.03.01 «Информационная безопасность»': 'https://abit.itmo.ru/bachelor/rating_rank/all/266/',
 '11.03.02 «Инфокоммуникационные технологии и системы связи»': 'https://abit.itmo.ru/bachelor/rating_rank/all/267/',
 '11.03.03 «Конструирование и технология электронных средств»': 'https://abit.itmo.ru/bachelor/rating_rank/all/268/',
 '12.03.01 «Приборостроение»': 'https://abit.itmo.ru/bachelor/rating_rank/all/269/',
 '12.03.02 «Оптотехника»': 'https://abit.itmo.ru/bachelor/rating_rank/all/270/',
 '1

In [6]:
fac_name = '01.03.02 «Прикладная математика и информатика»'

In [23]:
def get_discip(columns):
    x = list(columns)
    return x[x.index("Вид")+1:x.index('ЕГЭ+ИД')]

def parse_ifmo(fac_name):

    url = fac_list.get(fac_name)
    
    html = get_html_rec(url, 7)
        
    table = pd.read_html(html, header=1)[0]
    
    discip = get_discip(table.columns)

    date = BS(html).select_one(".static-page-rule > div:nth-child(1) > p:nth-child(5)").text[23:]

    print(date)

    header_str = BS(html).select_one(".static-page-rule > div:nth-child(1) > h1:nth-child(1)").text

    def parse_K(s):
        spn = re.search(r",К:\d+", s).span()
        num = s[spn[0]:spn[1]][3:]
        print("num is " + num)
        return int(num)

    free_places = parse_K(header_str)
    num_olymp = table[table["Условие поступления"] == "без вступительных испытаний"].shape[0]

    if num_olymp < free_places:
        free_places -= num_olymp
    else:
        free_places = 0
    
    scores = table[table["Условие поступления"] == 'по общему конкурсу']["ЕГЭ+ИД"]
    scores = [ int( re.search(r"\d+", str(i)).group(0) ) if str(i)[:3] != "nan" else 0 for i in scores]
    
    scores = np.sort(scores)[::-1]
#     return (scores)

    code = re.search(r"\d{2}.\d{2}.\d{2}",header_str).group(0)
    date_updated = datetime.strptime(date, "%H:%M:%S %d.%m.%Y")

    print(code)
    
    try:
        prev_years_list = list(
            prev_years[prev_years["Шифр"] == code][ 
                ["Проходной балл (ЕГЭ+ИД) 2017", "Проходной балл (ЕГЭ+ИД) 2018", "Проходной балл (ЕГЭ+ИД) 2019"] 
            ].values[0].astype(int)
        )
    except IndexError:
        prev_years_list = [-1, -1, -1]
        
    budjet_scores = scores[:free_places]
    
    
    
    if free_places == 0:
        last_score = 999
    else:
        last_score = budjet_scores[-1] if len(scores) >= free_places else 0
    

    res = {
        "fac_name" : fac_name,
        "date_updated" : f"{str(date_updated)}",
        "scores" : budjet_scores,
        "last_score" : last_score,
        "free_places" : free_places,
        "olymp_cnt" : num_olymp,
        "prev_years17" : prev_years_list[0] if prev_years_list[0] >= 0 else -1,
        "prev_years18" : prev_years_list[1] if prev_years_list[1] >= 0 else -1,
        "prev_years19" : prev_years_list[2] if prev_years_list[2] >= 0 else -1,
    }
    return res, discip

In [24]:
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
from time import sleep

In [27]:
parse_ifmo("09.03.01 «Информатика и вычислительная техника»")

00:00:17 05.08.2020
num is 40
09.03.01


({'fac_name': '09.03.01 «Информатика и вычислительная техника»',
  'date_updated': '2020-08-05 00:00:17',
  'scores': array([310, 304, 302, 301, 300, 299, 298, 298, 297, 296, 296, 296, 296,
         296, 296, 296, 296, 295, 295, 295, 294, 294, 294, 293, 293, 293,
         293, 292, 292, 292, 292, 292, 292, 292, 292, 292]),
  'last_score': 292,
  'free_places': 36,
  'olymp_cnt': 4,
  'prev_years17': 272,
  'prev_years18': 272,
  'prev_years19': 284},
 ['Математика', 'Русский язык', 'Информатика'])

In [28]:
Parsed_ifmo = []

facults = {}
for name, link in tqdm(fac_list.items() ):
    data, discip = parse_ifmo(name)
    
    print(name, link, discip)
    
    Parsed_ifmo.append(
        
        data
    
    )
    
    facults[name] = discip
    
    sleep(2)

  0%|          | 0/29 [00:00<?, ?it/s]

00:00:17 05.08.2020
num is 96
01.03.02
01.03.02 «Прикладная математика и информатика» https://abit.itmo.ru/bachelor/rating_rank/all/261/ ['Математика', 'Русский язык', 'Информатика']


  3%|▎         | 1/29 [00:03<01:31,  3.28s/it]

00:00:17 05.08.2020
num is 40
09.03.01
09.03.01 «Информатика и вычислительная техника» https://abit.itmo.ru/bachelor/rating_rank/all/262/ ['Математика', 'Русский язык', 'Информатика']


  7%|▋         | 2/29 [00:06<01:28,  3.27s/it]

...

 93%|█████████▎| 27/29 [01:19<00:06,  3.01s/it]

00:00:17 05.08.2020
num is 16
44.03.04
44.03.04 «Профессиональное обучение» https://abit.itmo.ru/bachelor/rating_rank/all/288/ ['Математика', 'Русский язык', 'Информатика']


 97%|█████████▋| 28/29 [01:22<00:02,  2.94s/it]

00:00:17 05.08.2020
num is 14
45.03.04
45.03.04 «Интеллектуальные системы в гуманитарной сфере» https://abit.itmo.ru/bachelor/rating_rank/all/289/ ['Математика', 'Русский язык', 'Иностранный язык']


100%|██████████| 29/29 [01:25<00:00,  2.95s/it]


In [29]:
pd.DataFrame(Parsed_ifmo)

,fac_name,date_updated,scores,last_score,free_places,olymp_cnt,prev_years17,prev_years18,prev_years19
0,01.03.02 «Прикладная математика и информатика»,2020-08-05 00:00:17,"[310, 310, 310, 309, 309, 309, 308, 308, 307, ...",300,28,68,309,283,302
1,09.03.01 «Информатика и вычислительная техника»,2020-08-05 00:00:17,"[310, 304, 302, 301, 300, 299, 298, 298, 297, ...",292,36,4,272,272,284
2,09.03.02 «Информационные системы и технологии»,2020-08-05 00:00:17,[],999,0,127,300,291,303
3,09.03.03 «Прикладная информатика»,2020-08-05 00:00:17,"[309, 302, 302, 300, 299, 298, 296, 296, 296, ...",292,17,2,285,282,288
4,09.03.04 «Программная инженерия»,2020-08-05 00:00:17,"[310, 310, 310, 309, 309, 306, 306, 305, 304, ...",302,15,72,282,284,289
5,10.03.01 «Информационная безопасность»,2020-08-05 00:00:17,"[308, 302, 301, 299, 299, 298, 297, 297, 296, ...",286,46,13,269,263,276
6,11.03.02 «Инфокоммуникационные технологии и си...,2020-08-05 00:00:17,"[296, 296, 292, 291, 290, 290, 290, 289, 288, ...",281,54,2,265,261,273
7,11.03.03 «Конструирование и технология электро...,2020-08-05 00:00:17,"[305, 287, 279, 277, 277, 275, 275, 273]",273,8,0,259,260,272
8,12.03.01 «Приборостроение»,2020-08-05 00:00:17,"[292, 284, 280, 279, 277, 276, 272, 271, 270, ...",267,14,0,245,253,262
9,12.03.02 «Оптотехника»,2020-08-05 00:00:17,"[293, 292, 283, 279, 272, 271, 269, 268, 268, ...",261,21,0,234,243,247


In [32]:
for i in Parsed_ifmo:
    i["scores"] =  list( i["scores"] )

In [33]:
import json

In [35]:
for i in facults.keys(): 
    for j in range(len(facults[i])): 
        if facults[i][j].startswith("Информатика"): 
            facults[i][j] = "Информатика и ИКТ" 

with open("disciplines.json", "w") as file:
    file.write( json.dumps( facults) )

In [29]:
Parsed_ifmo = (str(datetime.now()), Parsed_ifmo)

In [30]:
Parsed_ifmo

('2020-07-30 04:02:52.171399',
 [{'fac_name': '01.03.02 «Прикладная математика и информатика»',
   'date_updated': '2020-07-29 23:45:08',
   'scores': [310,
    309,
    307,
    306,
    306,
    306,
    306,
    304,
    304,
    303,
    302,
    302,
    301,
    301,
    300,
    300,
    299,
    299,
    299,
    298,
    298,
    298,
    298,
    297,
    297,
    296,
    296,
    296,
    296,
    296,
    295,
    295,
    295,
    295,
    295,
    294,
    294,
    294,
    294,
    293,
    293,
    293,
    292,
    292,
    292,
    292],
   'last_score': 292,
   'free_places': 46,
   'olymp_cnt': 50,
   'prev_years17': 309,
   'prev_years18': 283,
   'prev_years19': 302},
  {'fac_name': '09.03.01 «Информатика и вычислительная техника»',
   'date_updated': '2020-07-29 23:45:08',
   'scores': [310,
    302,
    299,
    298,
    298,
    297,
    296,
    296,
    296,
    296,
    295,
    294,
    294,
    294,
    293,
    292,
    292,
    292,
    292,
    292,
  

In [31]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [32]:
json_dump = json.dumps(Parsed_ifmo, cls=NumpyEncoder)

In [33]:
file_name = save_file(json_dump)

Saved to 2020-07-30 04:02:52.258638.json


In [34]:
!cat "{file_name}"

["2020-07-30 04:02:52.171399", [{"fac_name": "01.03.02 \u00ab\u041f\u0440\u0438\u043a\u043b\u0430\u0434\u043d\u0430\u044f \u043c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u043a\u0430 \u0438 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0442\u0438\u043a\u0430\u00bb", "date_updated": "2020-07-29 23:45:08", "scores": [310, 309, 307, 306, 306, 306, 306, 304, 304, 303, 302, 302, 301, 301, 300, 300, 299, 299, 299, 298, 298, 298, 298, 297, 297, 296, 296, 296, 296, 296, 295, 295, 295, 295, 295, 294, 294, 294, 294, 293, 293, 293, 292, 292, 292, 292], "last_score": 292, "free_places": 46, "olymp_cnt": 50, "prev_years17": 309, "prev_years18": 283, "prev_years19": 302}, {"fac_name": "09.03.01 \u00ab\u0418\u043d\u0444\u043e\u0440\u043c\u0430\u0442\u0438\u043a\u0430 \u0438 \u0432\u044b\u0447\u0438\u0441\u043b\u0438\u0442\u0435\u043b\u044c\u043d\u0430\u044f \u0442\u0435\u0445\u043d\u0438\u043a\u0430\u00bb", "date_updated": "2020-07-29 23:45:08", "scores": [310, 302, 299, 298, 298, 297, 296, 296, 296, 2

(Дата создания, [
    {
        'fac_name' : Название фака, 
        'date_updated': Дата с сайта(если есть) или нон есои нет, 
        'scores': [] Список баллов, 
        'last_score': Последний балл на бюджет, 
        'free_places': Количество мест , 
        'olymp_cnt': Количество бви, 
        'prev_years17': проходной за 17 год, 
        'prev_years18': проходной за 18 год, 
        'prev_years19':  проходной за 19 год,
    },
    
])